In [1]:
from scipy.spatial.transform import Rotation
from isaacgym.torch_utils import to_torch

from rrc.env import fop, make_env
from trifinger_simulation.tasks.move_cube import Pose
import multiprocessing as mp
import numpy as np
import torch

Importing module 'gym_38' (/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.10.0+cu102
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py38_cu102 as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py38_cu102/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


In [3]:
# %pdb on
env = make_env.env_fn_generator(diff=1, env_cls='robot_wrench_env', flatten_goal=True,
                               use_traj_opt=False)()
obs = env.reset()
obs, r, d ,i = env.step(np.zeros(6))

In [4]:
acscale = np.concatenate([env.force_factor, [env.torque_factor]*3])
cp_wf = env.get_cp_wf_list(None, obj_pose=Pose(obs['achieved_goal'][:3], obs['achieved_goal'][3:]), use_actual_cp=True)
cp_list = []
for cpf in cp_wf:
    if cpf is None:
        cp_list.append(np.array([0]*6 + [1]))
    else:
        cp_list.append(np.concatenate(cpf))
cp_v = np.array(cp_list)
n = 1024
cp_list = np.array([cp_v.flatten() for i in range(n)]).astype('float32')

In [5]:
cp_v

array([[-8.43550523e-02,  2.75191342e-03,  3.69284796e-02,
         3.18449096e-06,  2.50979308e-06,  7.85395725e-01,
        -6.18993987e-01],
       [-5.79314364e-02, -3.72151363e-02,  3.58668177e-02,
        -6.24734813e-03,  7.40228387e-04,  9.93033843e-01,
         1.17661418e-01],
       [-1.21413341e-01, -2.33746197e-02,  3.95698661e-02,
         2.82240769e-03,  2.13487065e-02,  1.31034274e-01,
        -9.91143928e-01]])

In [5]:
env.fop.get_grasp_matrix(to_torch(cp_v))

/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/torch_utils.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=dtype, device=device, requires_grad=requires_grad)


pybullet build time: Jul  8 2020 18:24:53


tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  6.8587e-01, -2.5774e-01,
         -6.8055e-01,  1.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  3.5177e-01,  9.3609e-01,
         -1.2811e-07,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  6.3705e-01, -2.3939e-01,
          7.3270e-01,  0.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -8.4986e-02, -3.8203e-02,
         -7.1183e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  3.0852e-02, -1.1594e-02,
         -6.0992e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  7.4463e-02, -4.2034e-03,
         -6.6116e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00]], device='cuda:0')

In [16]:
%time x = apply(cvxlayer.get_grasp_matrix, to_torch(cp_list))

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [17]:
cp_list_t = to_torch(cp_list)
np.apply_along_axis(torch.is_tensor, 1, cp_list)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

> /scr1/.pyenv/versions/rlgpu/lib/python3.8/site-packages/torch/_tensor.py(643)__array__()
    641             return handle_torch_function(Tensor.__array__, (self,), self, dtype=dtype)
    642         if dtype is None:
--> 643             return self.numpy()
    644         else:
    645             return self.numpy().astype(dtype, copy=False)



ipdb>  q


In [30]:
cvxlayer = fop.BatchForceOptProblem(device='cuda:0')

In [31]:
des_wrench_t = torch.tensor([env.action_space.sample().astype('float32') * acscale for _ in range(n)], dtype=torch.float).to(cvxlayer.device)

In [32]:
cp_list_t = torch.as_tensor(cp_list).to(cvxlayer.device)

In [34]:
%%time 
lamb = cvxlayer(des_wrench_t, cp_list)

CPU times: user 9.12 s, sys: 1.34 s, total: 10.5 s
Wall time: 6.45 s


In [15]:
lamb.shape

torch.Size([1024, 9])

In [4]:
cp_wf = [(np.array([-0.04190509,  0.00978352,  0.03772653]),
np.array([ 7.71566743e-06,  6.29104656e-06,  7.75028033e-01, -6.31926853e-01])),
 (np.array([-0.013862  , -0.0309698 ,  0.03706019]),
  np.array([ 2.23345856e-05, -2.27164750e-06,  9.94867341e-01,  1.01187815e-01])),
 (np.array([-0.07742409, -0.01736268,  0.03893098]),
  np.array([-2.27164750e-06, -2.23345856e-05,  1.01187815e-01, -9.94867341e-01]))]


In [8]:
np.array([np.concatenate(cpwf) for cpwf in cp_wf]).shape

(3, 7)

In [8]:
R_o_2_wf = Rotation.from_quat(obs['achieved_goal'][3:])
cp_list = []
for cp in cp_of:
    if cp is not None:
        c_pos, c_ori = cp
        c_pos_wf = R_o_2_wf.as_matrix() @ c_pos
        c_ori_wf = R_o_2_wf * Rotation.from_quat(c_ori)
        c_ori_wf = c_ori_wf.as_quat()
        cp_list.append((c_pos_wf, c_ori_wf))
    else:
        cp_list.append(None)

```python
        if use_actual_cp and self._current_contact_pos is not None:
            R_o_2_wf = Rotation.from_quat(obj_pose.orientation)
            R_w_2_of = R_o_2_wf.inv().as_matrix()
            flip_z = Rotation.from_rotvec([0, 0, np.pi]).as_matrix()
            for pos_wf, ori in zip(
                self._current_contact_pos, self._current_contact_ori
            ):
                if ori is None:
                    cp_list.append(None)
                else:
                    cp_quat = Rotation.from_matrix(R_w_2_of @ flip_z @ ori).as_quat()
                    R = Rotation.from_quat(obj_pose.orientation) * Rotation.from_quat(
                        cp_quat
                    )
                    quat_cp_2_w = R.as_quat()
                    cp_list.append((pos_wf, quat_cp_2_w))
        else:
            R_o_2_wf = Rotation.from_quat(obj_pose.orientation)
            for cp_param in cp_params:
                if cp_param is not None:
                    cp_of = c_utils.get_cp_of_from_cp_param(cp_param)
                    pos_wf = c_utils.get_wf_from_of(cp_of.pos_of, obj_pose)
                    R = R_o_2_wf * Rotation.from_quat(cp_of.quat_of)
                    quat_cp_2_w = R.as_quat()
                    # pos_wf = R_o_2_wf.as_matrix() @ cp_of.pos_of
                    cp_list.append((pos_wf, quat_cp_2_w))
```

In [9]:
cp_list

[(array([-0.04189374,  0.00980485, -0.59420026]),
  array([ 7.71566743e-06,  6.29104656e-06,  7.75028033e-01, -6.31926853e-01])),
 (array([-0.01385064, -0.03094847, -0.5948666 ]),
  array([ 2.23345856e-05, -2.27164750e-06,  9.94867341e-01,  1.01187815e-01])),
 (array([-0.07741273, -0.01734135, -0.5929958 ]),
  array([-2.27164750e-06, -2.23345856e-05,  1.01187815e-01, -9.94867341e-01]))]

In [17]:
prob = fop.BatchForceOptProblem()
G2 = prob.get_grasp_matrix(cp_list)

ValueError: cannot reshape array of size 21504 into shape (3,7)

In [ ]:
op = obs['achieved_goal']
obj_pose = np.asarray([op for _ in range(10)])
cp_wf = [cp_wf for _ in range(10)]

In [41]:
env.pinocchio_utils.forward_kinematics(env.unwrapped.prev_observation['observation']['position'])

[[0.02326169384260848, 0.06405488240915017, 0.04545479707254271],
 [0.029158603370715852, -0.018415214062422455, 0.04492667906354321],
 [-0.04664714912920895, 0.0026628426942095426, 0.0449847631644697]]

In [42]:
[x[0] for x in cp_wf[0][0]]

[array([0.02032874, 0.05563401, 0.85349421]),
 array([ 0.02290265, -0.01268787,  0.85237941]),
 array([-0.03887879,  0.00640975,  0.85346884])]

In [19]:
env.close()

In [1]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])

In [2]:
%%time 
bs = 2048
A_tch = torch.randn(bs, m, n, requires_grad=True)
b_tch = torch.randn(bs, m, requires_grad=True)

# solve the problem
solution, = cvxpylayer(A_tch, b_tch)

# compute the gradient of the sum of the solution with respect to A, b
solution.sum().backward()

CPU times: user 17.9 s, sys: 3.73 s, total: 21.6 s
Wall time: 15.1 s
